In [1]:
from langchain_community.document_loaders import PyPDFLoader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
import numpy as np
import nltk
import math
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jayantparakh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jayantparakh/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jayantparakh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
stopword = set(stopwords.words('english'))

In [3]:
loader = PyPDFLoader(file_path='Report.pdf',extract_images=True)

In [4]:
docs = loader.lazy_load()

In [5]:
print(docs)

<generator object PyPDFLoader.lazy_load at 0x130d7adc0>


In [6]:
docs1=[]
for doc in docs:
    docs1.append(doc)

In [7]:
def preprocess(content):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    twords = word_tokenize(text=content.lower())
    twords = [lemmatizer.lemmatize(word) for word in twords if word.isalpha() and word not in stopword]
    word_freq = Counter(twords)
    word_freq_dict = dict(word_freq)
    return word_freq_dict

In [8]:
wordfreqs={}
for i in docs1:
    wordfreqs[i.metadata['page']]=preprocess(i.page_content)

In [9]:
def pagerank(query):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    twords = word_tokenize(text=query.lower())
    twords = [lemmatizer.lemmatize(word) for word in twords if word.isalpha() and word not in stopword]
    sum1 = {word:0 for word in twords}
    pagefreq = {word:0.0001 for word in twords}
    for word in twords:
        for i in wordfreqs:
            if word in wordfreqs[i].keys():
                sum1[word]+=wordfreqs[i][word]
                pagefreq[word]+=1
    tf = {}
    for word in twords:
        tf[word] = {}          
        for i in wordfreqs:
            if word in wordfreqs[i].keys():
                tf[word][i]=(wordfreqs[i][word]/sum1[word])
            else:
                tf[word][i]=0


    idf = {}
    totalpages = len(wordfreqs.keys())
    for word in twords:
        idf[word] = math.log(totalpages/pagefreq[word])

    tfidfscore = {}
    pagerankscore = {i:0 for i in range(totalpages)}
    for word in twords:
        tfidfscore[word]={}
        for i in range(totalpages):
             tfidfscore[word][i]=tf[word][i]*idf[word]
             pagerankscore[i]+=tfidfscore[word][i]
    return pagerankscore


In [10]:
ranks = pagerank("oracle internship automation spring boot framework is am are was were has")

In [11]:
ranks

{0: 0.13095888851712675,
 1: 0.26902977546087714,
 2: 0.9547612630071851,
 3: 0.08696362169794256,
 4: 0.34141921851163437,
 5: 0.2647317092739687,
 6: 0.1313654249572978,
 7: 0.16977951506925784,
 8: 0.2627308499145956,
 9: 0.1461846539181808,
 10: 0.05614109396167742,
 11: 0.008003437437492513,
 12: 0.5121057896752825}

In [12]:
scores = ranks
sorted_indices = sorted(scores, key=scores.get, reverse=True)
rankings = {index: rank + 1 for rank, index in enumerate(sorted_indices)}
print(rankings)

{2: 1, 12: 2, 4: 3, 1: 4, 5: 5, 8: 6, 7: 7, 9: 8, 6: 9, 0: 10, 3: 11, 10: 12, 11: 13}
